<a href="https://colab.research.google.com/github/juhumkwon/DataMining/blob/main/YOLO_%EB%AA%A8%EB%8D%B8(COCO%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%99%EC%8A%B5_%EC%B6%94%EB%A1%A0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

# 간단한 YOLO 모델 정의
class SimpleYOLO(tf.keras.Model):
    def __init__(self, num_classes):
        super(SimpleYOLO, self).__init__()
        self.backbone = tf.keras.Sequential([
            tf.keras.layers.Conv2D(16, (3, 3), strides=1, padding="same", activation="relu"),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(32, (3, 3), strides=1, padding="same", activation="relu"),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Conv2D(64, (3, 3), strides=1, padding="same", activation="relu"),
            tf.keras.layers.MaxPooling2D((2, 2)),
        ])
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.Sequential([
            tf.keras.layers.Dense(256, activation="relu"),
            tf.keras.layers.Dense(num_classes * 5, activation="linear")  # 4 bbox + 1 confidence per class
        ])

    def call(self, inputs):
        x = self.backbone(inputs)
        x = self.flatten(x)
        return self.fc(x)

# COCO 데이터셋 로드 및 전처리
def preprocess_coco(image, annotations):
    image = tf.image.resize(image, (64, 64)) / 255.0  # 이미지를 64x64 크기로 리사이즈하고 정규화
    # annotations를 모델에 맞게 변환 필요 (예제에서는 생략)
    return image, tf.zeros((80 * 5,))  # 80 클래스 기준, 더미 타겟 반환

def get_coco_dataset(batch_size=8):
    dataset, _ = tfds.load("coco/2017", split="train", data_dir="./tensorflow_datasets", with_info=True)
    dataset = dataset.map(lambda x: preprocess_coco(x["image"], x["objects"]))
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# 학습된 모델로 추론하기
def predict_with_model(model, image):
    image_resized = tf.image.resize(image, (64, 64)) / 255.0  # 이미지 전처리
    image_resized = tf.expand_dims(image_resized, axis=0)  # 배치 차원 추가
    predictions = model(image_resized)  # 모델 추론
    return predictions.numpy()

# 바운딩 박스 시각화
def visualize_predictions(image, predictions, num_classes):
    plt.figure(figsize=(6, 6))
    plt.imshow(image.numpy().astype("uint8"))
    plt.axis("off")

    # 바운딩 박스와 confidence 추출 (예시: 첫 번째 클래스)
    for i in range(num_classes):
        confidence = predictions[0, i * 5 + 4]  # Confidence 값
        if confidence > 0.5:  # Confidence가 일정 이상인 경우만 표시
            x, y, w, h = predictions[0, i * 5:i * 5 + 4]  # 바운딩 박스 좌표
            x1, y1, x2, y2 = x - w / 2, y - h / 2, x + w / 2, y + h / 2
            plt.gca().add_patch(plt.Rectangle((x1 * 64, y1 * 64), w * 64, h * 64,
                                              edgecolor="red", facecolor="none", linewidth=2))
            plt.text(x1 * 64, y1 * 64, f"Class {i} ({confidence:.2f})", color="red")
    plt.show()

# 모델 학습 루프
def train_model():
    num_classes = 80  # COCO 데이터셋 클래스 수
    model = SimpleYOLO(num_classes)

    # 손실 함수 및 옵티마이저 설정
    loss_fn = tf.keras.losses.MeanSquaredError()
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    # 데이터셋 로드
    dataset = get_coco_dataset()

    # 모델 컴파일 및 학습
    model.compile(optimizer=optimizer, loss=loss_fn)
    model.fit(dataset, epochs=5)

    return model

# 실행 및 추론
if __name__ == "__main__":
    # 1. 모델 학습
    model = train_model()

    # 2. COCO 데이터셋에서 테스트 이미지 로드
    dataset, _ = tfds.load("coco/2017", split="validation", data_dir="./tensorflow_datasets", with_info=True)
    test_sample = next(iter(dataset))  # 첫 번째 샘플 선택
#    test_image = test_sample["image"]
    test_image = test_sample["bird.jpg"]

    # 3. 모델 추론
    predictions = predict_with_model(model, test_image)

    # 4. 결과 시각화
    visualize_predictions(test_image, predictions, num_classes=80)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]